In [1]:
import numpy
import scipy
import netCDF4
import matplotlib.pyplot as mp
import matplotlib.ticker
import matplotlib.colors
import scipy.stats
import pandas
import itertools
from mpl_toolkits import basemap
import mpl_toolkits.axes_grid1
import random
import datetime
import os
import time

degree_sign = u'\u00B0'
mp.rcParams.update({'mathtext.default': 'regular'})
mp.rcParams['hatch.color'] = '0.5'

% matplotlib inline

In [2]:
working_dir = '/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/npy_files/'
#save_dir = '/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/calculations/npy_files/'
#latlon_indices = numpy.load(working_dir + 'ccal_latlon_indices_array.npy'); region='ccal'
#latlon_indices = numpy.load(working_dir + 'ncal_latlon_indices_array.npy'); region='ncal'
#latlon_indices = numpy.load(working_dir + 'scal_latlon_indices_array.npy'); region='scal'

PRECT_lat = numpy.load('/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/npy_files/PRECT_lat.npy')
PRECT_lon = numpy.load('/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/npy_files/PRECT_lon.npy')

PRECT_nlat = 26
PRECT_nlon = 25

latlon_indices = list(itertools.product(range(PRECT_nlat), range(PRECT_nlon)))
region = 'whole_domain'

In [3]:
LA_lat = 34.0522
LA_lon = 118.2437 # deg west
LA_lon = 180. + (180-LA_lon)

Oroville_dam_lat = 39.5380
Oroville_dam_lon = 121.4831 # deg west
Oroville_dam_lon = 360 - Oroville_dam_lon

SF_lat = 37.7749
SF_lon = 122.4194
SF_lon = 360 - SF_lon

# Open coastal lat/lon indices

In [4]:
cali_coast_latlon_indices = numpy.load('/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/npy_files/coastal_latlon_array_indices_3x.npy')
cali_coast_latlon_indices_zip = [i for i in zip(cali_coast_latlon_indices[:,0], cali_coast_latlon_indices[:,1])]

cali_coast_latlon_indices_3d = cali_coast_latlon_indices.reshape((11,3,2))
#print(cali_coast_latlon_indices_3d[0,:,:])

In [5]:
def geo_idx(dd, dd_array):
   """
     search for nearest decimal degree in an array of decimal degrees and return the index.
     np.argmin returns the indices of minium value along an axis.
     so subtract dd from all values in dd_array, take absolute value and find index of minium.
    """
   geo_idx = (numpy.abs(dd_array - dd)).argmin()
   return geo_idx

In [6]:
LA_lat_idx = geo_idx(LA_lat, PRECT_lat)
LA_lon_idx = geo_idx(LA_lon, PRECT_lon)

SF_lat_idx = geo_idx(SF_lat, PRECT_lat)
SF_lon_idx = geo_idx(SF_lon, PRECT_lon)

OD_lat_idx = geo_idx(Oroville_dam_lat, PRECT_lat)
OD_lon_idx = geo_idx(Oroville_dam_lon, PRECT_lon)

#REGION_lat_idx, REGION_lon_idx = SF_lat_idx, SF_lon_idx; REGION_NAME = 'SF'
REGION_lat_idx, REGION_lon_idx = LA_lat_idx, LA_lon_idx; REGION_NAME = 'LA'
#REGION_lat_idx, REGION_lon_idx = OD_lat_idx, OD_lon_idx; REGION_NAME = 'OD'

In [7]:
window=30

# open data sets

In [8]:
year_start_pic = 402 #time_subsets[chunk,0]
year_end_pic = 2200 #time_subsets[chunk,1]

# create season strings
years_pic = numpy.arange(year_start_pic, year_end_pic+1, 1).astype(numpy.int)
half_years_pic = numpy.arange(year_start_pic+0.75, year_end_pic, 1)
#season_strings_pic = numpy.empty(years.size-1, dtype=numpy.str)

season_strings_pic = [str(years_pic[i])+'-'+str(years_pic[i+1]) for i in range(years_pic.size-1)]
member_strings_pic = ['{:03d}'.format(i) for i in range(1,36)]

n_seasons_pic=year_end_pic-year_start_pic

In [9]:
ensemble_members = numpy.hstack((numpy.arange(1,36), numpy.arange(101,106)))
ensemble_names = ['{:03d}'.format(i) for i in ensemble_members]

year_start = 1920
year_end = 2100

year_start_list = numpy.arange(1920,2070)
year_end_list = numpy.arange(1950,2100)
year_middle_list = year_start_list+15

# create season strings
years = numpy.arange(year_start, year_end+1, 1).astype(numpy.int)
half_years_hist_rcp = numpy.arange(year_start+0.75, year_end, 1)
season_strings_hist_rcp = [str(i)+'-'+str(i+1) for i in range(year_start,year_end)]

thirty_yr_strings_hist_rcp = [str(year_start_list[i])+'-'+str(year_end_list[i]) for i in range(year_start_list.size)]
member_strings_hist_rcp = ['{:03d}'.format(i) for i in range(1,36)]

n_seasons_hist_rcp=year_end-year_start

# Create dictionary to store everything

In [10]:
events_dict = {}

# do seasonal whiplash calculations

In [44]:
filename = '/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/calcs_and_plots/whiplash/calculations/time_series_hist_rcp_dry20_wet80_events.nc'
ncfile = netCDF4.Dataset(filename)
hist_rcp_dry_events = ncfile.variables['all_dry_events'][:]
hist_rcp_wet_events = ncfile.variables['all_wet_events'][:]
hist_rcp_time_var = ncfile.variables['time']
hist_rcp_time_data = ncfile.variables['time'][:]

In [45]:
hist_rcp_time_datetime = netCDF4.num2date(hist_rcp_time_data, units=hist_rcp_time_var.units, calendar='standard')

In [47]:
hist_rcp_time_datetime[0]

datetime.datetime(1921, 1, 15, 0, 0)

In [29]:
filename = '/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/calcs_and_plots/whiplash/calculations/seasonal_whiplash_PIC_low_to_high.nc'
ncfile = netCDF4.Dataset(filename, 'r', 'NetCDF4')
whiplash_events_pic = ncfile.variables['whiplash_events'][:]
pic_time_var = ncfile.variables['time']
pic_time_data = ncfile.variables['time'][:]

filename = '/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/calcs_and_plots/whiplash/calculations/seasonal_whiplash_hist_rcp_low_to_high.nc'
ncfile = netCDF4.Dataset(filename, 'r', 'NetCDF4')
whiplash_events_hist_rcp = ncfile.variables['seasonal_total'][:]
hist_rcp_time_var = ncfile.variables['time']
hist_rcp_time_data = ncfile.variables['time'][:]

In [39]:
pic_time_datetime = netCDF4.num2date(pic_time_data, pic_time_var.units, 'standard')
hist_rcp_time_datetime = netCDF4.num2date(hist_rcp_time_data, hist_rcp_time_var.units, 'standard')

In [40]:
hist_rcp_time_datetime[-1]

datetime.datetime(2100, 1, 15, 0, 0)

In [22]:
whiplash_count_pic = numpy.sum(whiplash_events_pic==1, axis=0)
whiplash_count_pic_mean = numpy.array(whiplash_count_pic)/1798.

# sum hist+rcp across ensemble members, then divide by 40
whiplash_count_hist_rcp = numpy.sum(whiplash_events_hist_rcp==1, axis=1)
whiplash_count_hist_rcp_mean = numpy.array(whiplash_count_hist_rcp)/40.

whiplash_ratios_change = 100*numpy.array((whiplash_count_hist_rcp==1) - whiplash_count_pic_mean)/(whiplash_count_pic_mean)
whiplash_ratios_mean = 100*whiplash_count_hist_rcp_mean/whiplash_count_pic_mean
whiplash_ratios = whiplash_count_hist_rcp/whiplash_count_pic_mean
#whiplash_ratios = numpy.array()

In [13]:
hist_rcp_counts_ALL_LATLON_smooth = numpy.zeros((whiplash_count_hist_rcp.shape))
for i in range(PRECT_nlat):
    for j in range(PRECT_nlon):
        hist_rcp_counts_ALL_LATLON_smooth[:,i,j] = pandas.Series(whiplash_count_hist_rcp[:,i,j]).rolling(window=window, center=True).mean()

pic_counts_ALL_LATLON_equivalent = whiplash_count_pic/(1798/40)
hist_rcp_counts_ALL_LATLON_smooth_norm = numpy.zeros((whiplash_count_hist_rcp.shape))
for i in range(PRECT_nlat):
    for j in range(PRECT_nlon):
        hist_rcp_counts_ALL_LATLON_smooth_norm[:,i,j] = (hist_rcp_counts_ALL_LATLON_smooth[:,i,j] - pic_counts_ALL_LATLON_equivalent[i,j])/pic_counts_ALL_LATLON_equivalent[i,j] * 100.

hist_rcp_counts_ALL_LATLON_smooth_ratios = numpy.zeros((whiplash_count_hist_rcp.shape))
for i in range(PRECT_nlat):
    for j in range(PRECT_nlon):
        hist_rcp_counts_ALL_LATLON_smooth_ratios[:,i,j] = hist_rcp_counts_ALL_LATLON_smooth[:,i,j]/pic_counts_ALL_LATLON_equivalent[i,j]

In [14]:
count_ratio_2d = hist_rcp_counts_ALL_LATLON_smooth_ratios[-15,:,:]
print(count_ratio_2d.shape)

(26, 25)


In [15]:
hist_rcp_counts_ALL_LATLON_smooth_ratios = hist_rcp_counts_ALL_LATLON_smooth_ratios.transpose(1,2,0)
events_dict['whiplash_pic_count'] = whiplash_count_pic
events_dict['whiplash_2d_frequency_ratios_30yr_avg'] = hist_rcp_counts_ALL_LATLON_smooth_ratios[:,:,-15]

In [16]:
print(hist_rcp_counts_ALL_LATLON_smooth_ratios.shape)

(26, 25, 180)


In [20]:
events_dict['whiplash_2d_frequency_ratios_30yr_avg'].shape

(26, 25)

# save netcdf file for maps

# Save time-latitude files